<a href="https://colab.research.google.com/github/M-SAAD-BIN-MAZHAR/QuestioningToolUsingRAG/blob/main/QuestioningToolUsingRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install langchain langchain-community
!pip -q install pypdf
!pip -q install sentence_transformers
!pip install openai
!pip install huggingface_hub
!pip install tiktoken
!pip install tokenizers
!pip install faiss-cpu
!pip -q install unstructured
!pip install numpy==1.24.4
!pip install nltk==3.9.1

In [ ]:
import sys
import os
import textwrap
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface import ChatHuggingFace,HuggingFaceEndpoint
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQAWithSourcesChain



In [ ]:
pip install langchain-huggingface


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

In [ ]:

os.environ['HUGGINGFACEHUB_API_TOKEN'] = ""

In [ ]:
URLs=[
    'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb',
    'https://www.mosaicml.com/blog/mpt-7b',
    'https://stability.ai/blog/stability-ai-launches-the-first-of-its-stablelm-suite-of-language-models',
    'https://lmsys.org/blog/2023-03-30-vicuna/'

]

In [ ]:
loaders=UnstructuredURLLoader(urls=URLs)
data=loaders.load()

In [ ]:
data

In [ ]:
text_splitter=CharacterTextSplitter(separator='\n',
                                    chunk_size=1000,
                                    chunk_overlap=200)

In [ ]:
text_chunks=text_splitter.split_documents(data)

In [ ]:
len(text_chunks)

In [ ]:
text_chunks[1]

In [ ]:
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
llm=HuggingFaceEndpoint(
    repo_id='mistralai/Mistral-7B-Instruct-v0.3',
    task='text-generation'
)
model=ChatHuggingFace(llm=llm)

In [ ]:
print(embeddings.embed_query('hello world'))

In [ ]:
vector_store=FAISS.from_documents(text_chunks,embeddings)

In [ ]:
model.invoke("hello")

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate

# Define a simple prompt
prompt = ChatPromptTemplate.from_template(
    "Use the following context to answer the question.\n\n{context}\n\nQuestion: {input}"
)

# Create a document chain
document_chain = create_stuff_documents_chain(
    llm=model,
    prompt=prompt,
)

# Build the retrieval chain
chain = create_retrieval_chain(
    retriever=vector_store.as_retriever(),
    combine_docs_chain=document_chain,   # ✅ correct key in 0.3.x
)

# Run the chain
response = chain.invoke({"input": "What is LangChain?"})
print(response)


In [ ]:
results=chain.invoke({'input':'How good is Vicuna?'},return_only_outputs=True)

In [ ]:
results['answer']

In [ ]:
wrapped_text=textwrap.fill(results['answer'],width=500)

In [ ]:
wrapped_text

In [ ]:
while True:
  query=input(f'Prompt: ')
  if query=='exit' :
    print('Exiting')
    sys.exit()
  if query =='':
    continue
  result=chain.invoke('input',query)
  print(f'Response: {result["answer"]}')